In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# pre-trained digit classification model
model = load_model('mnist_cnn_model.h5')

# Preprocess the image
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, thresh = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV)
    return thresh

# Find contours (potential digits)
def find_digit_contours(preprocessed_image):
    contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

# Predict the class for each contour
def classify_digits(model, image, contours):
    digit_predictions = []
    for contour in contours:
        # Compute the bounding box of the contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Crop and resize the image around the contour
        digit = image[y:y+h, x:x+w]
        digit = cv2.resize(digit, (28, 28))
        digit = np.expand_dims(digit, axis=-1)
        digit = np.expand_dims(digit, axis=0).astype('float32') / 255
        
        # prediction
        prediction = model.predict(digit)
        predicted_digit = np.argmax(prediction, axis=1)[0]  # Get the index of max predicted value
        digit_predictions.append((predicted_digit, (x, y, w, h)))
        
    return digit_predictions

image_path = 'pic.jpg'  # Change this path to your actual image path
thresh_image = preprocess_image(image_path)
digit_contours = find_digit_contours(thresh_image)
predictions = classify_digits(model, thresh_image, digit_contours)

# Load the original image to draw on
original_image = cv2.imread(image_path)

for predicted_digit, (x, y, w, h) in predictions:
    cv2.rectangle(original_image, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Draw the bounding box
    cv2.putText(original_image, str(predicted_digit), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, (36,255,12), 1)  # Draw the predicted digit

# Save the image
cv2.imwrite('question_with_predictions.jpg', original_image)

print('Saved image with predictions to: question_with_predictions.jpg')
